In [1]:
from pprint import pprint

In [9]:
class SodokuCell:
    def __init__(self, row, col, value):
        self.row = row
        self.col = col
        self.value = value
        self.row_in_square = row % 3
        self.col_in_square = col % 3
        self.square = 3 * (row // 3) + (col // 3)

    def __repr__(self):
        return '[%d, %d] -> %s' % (self.row, self.col, self.value)


class SodokuField:
    @staticmethod
    def from_string(field_str):
        ''' initializes a sodoku from a textual representation of the field
            spaces and newlines are ignored, everything else that is not
            a digit is treated as an empty cell. cells are specified row-wise
            from top to bottom and from left to right
        '''
        field = [
            int(char) if char.isdigit() else None
            for char in field_str.replace(' ', '').replace('\n', '')
        ]
        return SodokuField(field)
        
    def __init__(self, field):
        ''' initializes a sodoku field with the given content
            field is a list with 81 items, either numbers or None, specified
            row-wise from top to bottom and from left to right
        '''
        assert len(field) == 81
        
        row = col = 0
        self.rows = [[] for _ in range(9)]
        self.cols = [[] for _ in range(9)]
        self.cells = {}
        for number in field:
            assert number is None or 1 <= number <= 9
            cell = SodokuCell(row, col, number)
            self.rows[row].append(cell)
            self.cols[col].append(cell)
            self.cells[row, col] = cell
            
            col += 1
            if col == 9:
                row += 1
                col = 0
        
        self.squares = [[
            self.cells[i + k, j + l]
            for k in range(3) for l in range(3)
        ] for i in range(0, 9, 3) for j in range(0, 9, 3)]
        
        self.fill_candidates()
    
    def pretty_print(self):
        res = []
        for i, row in enumerate(self.rows):
            if i % 3 == 0:
                res.append('+---------+---------+---------+\n')
                
            for j, x in enumerate(row):
                if j % 3 == 0:
                    res.append('|')
                res.append(' %s ' % (x.value or ' '))

            res.append('|\n')
        res.append('+---------+---------+---------+\n')
        
        print(''.join(map(str, res)))
    
    def related_cells(self, cell):
        ''' returns all the cells in the same row/column/square
            as the given cell
        '''
        yield from (c for c in self.rows[cell.row] if c != cell)
        yield from (c for c in self.cols[cell.col] if c != cell)
        yield from (c for c in self.squares[cell.square] if c != cell)
    
    def write(self, cell, value):
        ''' sets the value of a given cell, and updates the candidates accordingly
        '''
        assert not value or 1 <= value <= 9
        cell.value = value
        cell.candidates = set()
        
        if value is None:
            self.fill_candidates()
        else:
            for other_cell in self.related_cells(cell):
                other_cell.candidates.discard(value)
    
        return cell
    
    def step_fill_with_rules(self):
        ''' applies one step of rules, returning a list of filled cells
        '''
        # should not be necessary as long as everybody uses self.write
        # or calls this when modifying things manually
        #self.fill_candidates()
        
        yield from self.fill_only_solution()
        yield from self.fill_only_position()

    def fill_with_rules(self):
        ''' repeatedly applies rules until no more moves are possible
            returns the cells that were filled
        '''
        done = False
        while not done:
            done = True
            for cell in self.step_fill_with_rules():
                yield cell
                done = False
                if not self.is_valid():
                    return
            
    def do_fill_with_rules(self):
        return list(self.fill_with_rules())
    
    def do_step_fill(self):
        return list(self.step_fill_with_rules())
    
    def fill_candidates(self):
        ''' find the candidates that can fill each cell
            a number x is a candidate for a cell if x does not appear in the
            cell's row, column or square
            does NOT validate (e.g. cells with no value and no candidates)
        '''
        for cell in self.cells.values():
            if cell.value is not None:
                cell.candidates = set()
                continue
            
            candidates = set(range(1, 10))
            for other in self.related_cells(cell):
                if other.value in candidates:
                    candidates.remove(other.value)
            
            cell.candidates = candidates
        
        self.remove_single_stack_candidates_in_square()
    
    def remove_single_stack_candidates_in_square(self):
        ''' consider a single square, if a value x can only be placed
            within cells of the same row/column, then we can remove x
            from the candidates of cells in the same row/column but
            different square
        '''
        def remove_candidate_from_stack_cells_not_in_square(value, stack, square):
            for cell in stack:
                if cell.square != square:
                    cell.candidates.discard(value)
        
        for i, square in enumerate(self.squares):
            possible_positions = {i: set() for i in range(1, 10)}
            for cell in square:
                for cand in cell.candidates:
                    possible_positions[cand].add(cell)
            
            for value, positions in possible_positions.items():
                if len(set(cell.row for cell in positions)) == 1:
                    only_row = list(positions)[0].row
                    remove_candidate_from_stack_cells_not_in_square(
                        value, self.rows[only_row], i
                    )
                elif len(set(cell.col for cell in positions)) == 1:
                    only_col = list(positions)[0].col
                    remove_candidate_from_stack_cells_not_in_square(
                        value, self.cols[only_col], i
                    )
    
    def fill_only_solution(self):
        ''' fills a cell if there's only one possible candidate
        '''
        for cell in self.cells.values():
            if len(cell.candidates) == 1:
                self.write(cell, cell.candidates.pop())
                yield cell
    
    def fill_only_position(self):
        ''' fills a cell c with value x if there is no other cell
            in c's row/column/square that can contain x
        '''
        
        def fill_by_stack(stack):
            possible_positions = {i: set() for i in range(1, 10)}
            for cell in stack:
                for cand in cell.candidates:
                    possible_positions[cand].add(cell)

            for value, positions in possible_positions.items():
                if len(positions) == 1:
                    yield self.write(positions.pop(), value)
        
        yield from (cell for row in self.rows for cell in fill_by_stack(row))
        yield from (cell for col in self.cols for cell in fill_by_stack(col))
        yield from (cell for square in self.squares for cell in fill_by_stack(square))

    def brute_force(self):
        ''' solves by brute-forcing (in a depth-first fashion)
            uses as many rules as possible, then tries to set a value
            to an empty cell, and repeats.
            
            returns tuple success (boolean), list of moves (not incl. filled by rules)
            after return, the cells contain the solution of the last (valid) state reached
        '''
        
        # start by trying rules
        moves = list(self.fill_with_rules())
        valid = self.is_valid()
        if valid and self.is_solved():
            return True, []
        elif not valid:
            # invalid state reached, undo and return failure
            # do not use self.write to avoid filling candidates every time
            for other_cell in moves:
                other_cell.value = None
            self.fill_candidates()
            return False, None
        
        # if valid but not solved, examine empty cells
        # ordered by number of candidates (ascending)
        empty_cells = sorted(
            filter(lambda c: c.candidates, self.cells.values()),
            key=lambda c: -len(c.candidates)
        )
        
        for cell in empty_cells:
            for candidate in cell.candidates:
                self.write(cell, candidate)
                
                success, solution = self.brute_force()
                if success:
                    return True, [cell] + solution

                self.write(cell, None)

        
        return False, None
    
    def violations(self):
        for cell in self.cells.values():
            if cell.value is None:
                if len(cell.candidates) == 0:
                    yield cell
            else:
                for other_cell in self.related_cells(cell):
                    if other_cell.value == cell.value:
                        yield cell, other_cell
    
    def is_valid(self):
        return not any(self.violations())
    
    def is_solved(self):
        return all(
            c.value is not None for c in self.cells.values()
        ) and not any(self.violations())

In [10]:
fields = ['''
.....63..
.5..3....
.73.8...5
.8..5.2..
...7.....
..7...4.3
....7...1
.3..4..8.
96...2...
''', '''
.19..3..8
....18...
3...6..7.
7........
...154...
.63....4.
........9
9.182.5..
8....7..2
''', '''
..9.7....
.56..2.4.
.4..9.3..
..4.....9
9.1....2.
..7..5...
..8.....1
.1...36..
.....84..
''', '''
.8..1....
..7.6...4
...8.79..
....4..2.
.365..178
.........
.7...15.3
9.5....4.
6..2.....
''', '''
.4.13.8.6
.1.4.6.7.
8.....2..
.6...3..2
.......3.
.2...7..8
....6..9.
7..9.....
.....1..3
''', '''
.6......9
1.2......
....8.46.
..83..7.1
9..5.....
43..7...8
.9...7...
..1..48..
24......5
''', '''
..98.....
.47.5.8..
..3....9.
.........
...684.1.
...79..2.
.92...73.
..81.5...
7.......6
''']

for i, f in enumerate(fields):
    sf = SodokuField.from_string(f)
    assert sf.is_valid()
    solved, _ = sf.brute_force()
    if not solved:
        print(i, 'failed')
        sf.pretty_print()
        print(sf.is_valid())
    else:
        print(i, 'done')

0 done
1 done
2 done
3 done
4 done
5 done
6 done
